In [1]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [2]:
# First set the structure.
wc_model = BayesianModel([('Genetics','Performance'),
                         ('Practice','Performance'),
                         ('Performance','Offer')])

In [3]:
# Then set up the relationships (the CPDs)
genetics_cpd = TabularCPD(
                variable = "Genetics",
                variable_card = 2,
                values = [[.2,.8]])

In [4]:
practice_cpd = TabularCPD(
                variable = 'Practice',
                variable_card = 2,
                values = [[.7,.3]])

In [5]:
offer_cpd = TabularCPD(
                variable = 'Offer',
                variable_card = 2,    
                values = [[.95, .8, .5],
                          [0.05, .2, .5]],
                evidence = ['Performance'],
                evidence_card = [3])

In [7]:
wc_trials_cpd = TabularCPD(
                variable = 'Performance',
                variable_card = 3,
                values = [[.5,.8,.8,.9],
                          [.3,.15,.1,.08],
                          [.2,.05,.1,.02]],
                evidence = ['Genetics','Practice'],
                evidence_card = [2,2])

In [8]:
# Add the relationship to the models
wc_model.add_cpds(genetics_cpd, practice_cpd, offer_cpd, wc_trials_cpd)

In [9]:
wc_model.get_cpds()

[<TabularCPD representing P(Genetics:2) at 0x1a2aeb78d0>,
 <TabularCPD representing P(Practice:2) at 0x1a2aeb7ac8>,
 <TabularCPD representing P(Offer:2 | Performance:3) at 0x1a2aeb7f98>,
 <TabularCPD representing P(Performance:3 | Genetics:2, Practice:2) at 0x1a2aece6a0>]

In [10]:
wc_model.active_trail_nodes('Genetics')

{'Genetics': {'Genetics', 'Offer', 'Performance'}}

In [11]:
# Find Local independencies
wc_model.local_independencies('Genetics')

(Genetics _|_ Practice)

In [12]:
wc_model.local_independencies('Performance')

In [13]:
wc_model.get_independencies()

(Genetics _|_ Practice)
(Genetics _|_ Offer | Performance)
(Genetics _|_ Offer | Performance, Practice)
(Practice _|_ Genetics)
(Practice _|_ Offer | Performance)
(Practice _|_ Offer | Performance, Genetics)
(Offer _|_ Practice, Genetics | Performance)
(Offer _|_ Genetics | Performance, Practice)
(Offer _|_ Practice | Performance, Genetics)

In [14]:
# Making Inferences
# We can get probability distributions that are not explicitly spelled out in our graphs
from pgmpy.inference import VariableElimination

In [15]:
wc_infer = VariableElimination(wc_model)

In [17]:
prob_offer = wc_infer.query(variables = ['Offer'])
print(prob_offer)

Eliminating: Genetics: 100%|██████████| 3/3 [00:00<00:00, 479.17it/s]

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8898 |
+----------+--------------+
| Offer(1) |       0.1102 |
+----------+--------------+


In [20]:
# We can also get conditional probability distributions that take into account what we already know
prob_offer_bad_genes = wc_infer.query(
                                    variables = ['Offer'],
                                    evidence = {'Genetics':1})
print(prob_offer_bad_genes)

Eliminating: Practice: 100%|██████████| 2/2 [00:00<00:00, 715.02it/s]

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.9017 |
+----------+--------------+
| Offer(1) |       0.0983 |
+----------+--------------+


In [21]:
prob_offer_good_genes = wc_infer.query(
                                    variables = ['Offer'],
                                    evidence = {'Genetics':0})
print(prob_offer_good_genes)

Eliminating: Practice: 100%|██████████| 2/2 [00:00<00:00, 669.16it/s]

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8420 |
+----------+--------------+
| Offer(1) |       0.1580 |
+----------+--------------+


In [22]:
prob_offer_good_genes_did_practice = wc_infer.query(
                                        variables = ['Offer'],
                                        evidence = {'Genetics': 0,'Practice': 0})
print(prob_offer_good_genes_did_practice)

Eliminating: Performance: 100%|██████████| 1/1 [00:00<00:00, 643.99it/s]

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8150 |
+----------+--------------+
| Offer(1) |       0.1850 |
+----------+--------------+


In [23]:
wc_infer.map_query(variables = ['Offer'])

Eliminating: Genetics: 100%|██████████| 3/3 [00:00<00:00, 615.99it/s]


{'Offer': 0}

In [24]:
wc_infer.map_query(variables = ['Performance'])

Eliminating: Genetics: 100%|██████████| 3/3 [00:00<00:00, 846.54it/s]


{'Performance': 0}

In [25]:
wc_infer.map_query(variables = ['Genetics'])

Eliminating: Practice: 100%|██████████| 3/3 [00:00<00:00, 761.63it/s]


{'Genetics': 1}

In [26]:
wc_infer.map_query(variables = ['Practice'])

Eliminating: Genetics: 100%|██████████| 3/3 [00:00<00:00, 730.63it/s]


{'Practice': 0}